# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
!pip install tqdm

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
from tqdm import tqdm
import torch
from datetime import datetime
from unityagents import UnityEnvironment
import numpy as np
from ddpg_agent import Agent
from utils import append_to_file, write_json
from sklearn.model_selection import ParameterGrid
import json
from pprint import pprint

In [2]:
env = UnityEnvironment(file_name="YOUR_APP_HERE")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


## Train your agent based on a hyperparameter set

In [ ]:
def train(hypes, n_episodes=4000, print_every=100, solved_rewards_threshold=0.5):

    start_time = str(datetime.now())
    best_score = -1
    base_dir = f"./hypes/run_{start_time}"
    agent = Agent(state_size=state_size, action_size=action_size, random_seed=param["seed_start"], hypes=hypes)

    try:
        import os
        os.makedirs(base_dir + "/checkpoint")
        os.makedirs(base_dir + "/final")
    except:
        pass

    hypes = {
        "ACTOR": str(agent.actor_local).split("\n"),
        "CRITIC": str(agent.critic_local).split("\n")
    }
    hypes.update(agent.hypes)

    write_json(hypes, base_dir + "/hypes.json")

    scores_deque = deque(maxlen=100)

    scores = [] # actual score
    avg_scores = [] # 100-episode average scores
    pbar = tqdm(range(1, n_episodes + 1))
    for i_episode in pbar:
        pbar.set_description(f"Average Score: {np.mean(scores_deque):.4f}")

        # Reset env and agent
        env_info = env.reset(train_mode=True)[brain_name]
        agent.reset()
        score = np.zeros(num_agents)
        states = env_info.vector_observations

        while True:
            # Get the SARS tuple

            action = np.squeeze(np.array([agent.act(s, False) for s in states]))
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations
            reward = env_info.rewards
            done = env_info.local_done

            for i in range(2):
                agent.add_experiences(states[i], action[i], reward[i], next_state[i], done[i])

            # Sample and learn
            agent.learn()
            states = next_state
            score += np.array(reward)

            if any(done):
                break

        # The score is defined as the best score between agents
        agent_score = max(score)

        scores_deque.append(agent_score)
        scores.append(agent_score)

        agent.decay_noise()

        score_mean_tuple = (scores[-1], np.mean(scores_deque))
        append_to_file(str(score_mean_tuple), base_dir + "/run.txt")

        avg_scores.append(np.mean(scores_deque))

        if np.mean(scores_deque) > solved_rewards_threshold:
            print("The environment has been solved. Saving checkpoints")
            agent.save(base_dir + "/final")
            break

        if i_episode % print_every == 0:
            tqdm.write('Episode {}\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_deque)))
            if avg_scores[-1] > best_score:
                print("Best score has exceeded last average score {:.4f} vs {:.4f}".format(best_score, avg_scores[-1]))
                best_score = avg_scores[-1]

                agent.save(base_dir + "/checkpoint")

    return i_episode, base_dir


## Run your agent across multiple hyperparameters
* Change hypes_search to different values to see how your agent performs!

In [ ]:
def hypes_search():
    return {
        "LR_ACTOR": [1e-4],
        "LR_CRITIC": [1e-3, 5e-4],
        "GAMMA": [0.99],
        "TAU": [1e-2, 1e-3],
        "BUFFER_SIZE": [1e5, 1e6],
        "BATCH_SIZE": [128],
        "eps_decay": [0, 0.98],
        "network_size": [256, 128],
        "seed_start": [2, 10]
    }

param_grid = hypes_search()
grid_start = str(datetime.now())
for param in ParameterGrid(param_grid):
    best_score, location = train(param)
    param["env_solved_episodes"] = best_score
    param["output_path"] = location

    pprint(param)

    append_to_file(json.dumps(param), "hypes/" + grid_start + " hype_output.txt")
    # Add a break in case you want to just train one agent
    # break


# Watch the Agent

In [ ]:
## Load the fastest converging agent
best_base = "./hypes/run_2018-12-20 15:05:11.127599"
best_params_file = best_base + "/hypes.json"
best_params_models = best_base + "/final"
with open(best_params_file, "r") as f:
    best_params = json.load(f)
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2, hypes=best_params)
agent.load(best_params_models)

In [ ]:
from ddpg_agent import Agent
import torch


full_scores = []
for _ in range(5):
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    agent.reset()

    score = np.zeros(num_agents)
    while True:
        
        states = env_info.vector_observations
        actions = np.squeeze(np.array([agent.act(s, False) for s in states]))
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards        
        score += env_info.rewards
        dones = env_info.local_done
        if np.any(dones):
            break
    
    print(np.max(score))
